In [1]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

Definisco alcune funzioni utili.

In [2]:
REGIONI = ["Abruzzo",
           "Basilicata",
           "Calabria",
           "Campania",
           "Emilia_Romagna",
           "Friuli_Venezia_Giulia",
           "Lazio",
           "Liguria",
           "Lombardia",
           "Marche",
           "Molise",
           "Piemonte",
           "Puglia",
           "Sardegna",
           "Sicilia",
           "Toscana",
           "Trentino_Alto_Adige",
           "Umbria",
           "Valle_d_Aosta",
           "Veneto"]

In [3]:
def get_prov(regione):
    url = urllib.request.urlopen("http://www.immobiliare.it/prezzi-mq/"+regione+"/")
    c = str(url.read())
    url.close()
    del url
    html = BeautifulSoup(c, 'html.parser').find(id="heatMapTablePrezzi")
    del c
    html = BeautifulSoup(str(html), 'html.parser').find_all("a")
    prov = {"PROVINCIA": [], "URL": []} 
    for item in html:
        prov["PROVINCIA"].append(item.get_text())
        prov["URL"].append(item.get("href"))
    del html
    return prov

In [4]:
def get_comu(prov_url):
    url = urllib.request.urlopen("http://www.immobiliare.it"+prov_url)
    c = str(url.read())
    url.close()
    del url
    html = BeautifulSoup(c, 'html.parser').find(id="heatMapTablePrezzi")
    del c
    html = BeautifulSoup(str(html), 'html.parser').find_all("a")
    comu = {"COMUNE": [], "URL": []} 
    for item in html:
        comu["COMUNE"].append(item.get_text())
        comu["URL"].append(item.get("href"))
    del html
    return comu

In [5]:
def get_cap(comu_url):
    url = urllib.request.urlopen("http://www.immobiliare.it"+comu_url)
    c = str(url.read())
    url.close()
    del url
    html = BeautifulSoup(c, 'html.parser').find(id="heatmapTableCap")
    del c
    html = BeautifulSoup(str(html), 'html.parser').find_all("a")
    capl = {"CAP": [], "URL": []} 
    for item in html:
        capl["CAP"].append(item.get_text())
        capl["URL"].append(item.get("href"))
    del html
    return capl

In [ ]:
def getp(regione, provincia, comune, CAP, CAP_url):
    try: 
        url = urllib.request.urlopen("http://www.immobiliare.it" + CAP_url)
        c = str(url.read())
        url.close()
        html = BeautifulSoup(c, 'html.parser').find(id="heatmapTableStreet")
        del c
        if str(html) != "None":
            rows = BeautifulSoup(str(html.tbody), 'html.parser').find_all("tr")
            rows = [ [x.get_text() for x in r.find_all("td")] for r in rows ]
            df = [ [[r[j]][0] for r in rows] for j in [0, 1, 2] ]
            del rows
            n = len(df[0])
            df = pd.DataFrame({"REGIONE": [regione] * n, 
                               "PROVINCIA": [provincia] * n,
                               "COMUNE": [comune] * n, 
                               "CAP": [CAP] * n,
                               "INDIRIZZO": df[0], 
                               "VENDITA_EURO/MQ": df[1], 
                               "AFFITTO_EURO/MQ_AL_MESE": df[2]})
            del n
            return df.loc[:,["REGIONE", "PROVINCIA", "COMUNE", "CAP", "INDIRIZZO", "VENDITA_EURO/MQ", "AFFITTO_EURO/MQ_AL_MESE"]]
        else:
            df = pd.DataFrame({"REGIONE": [regione], 
                               "PROVINCIA": [provincia],
                               "COMUNE": [comune], 
                               "CAP": [CAP],
                               "INDIRIZZO": [None], 
                               "VENDITA_EURO/MQ": [None], 
                               "AFFITTO_EURO/MQ_AL_MESE": [None]})
            return df.loc[:,["REGIONE", "PROVINCIA", "COMUNE", "CAP", "INDIRIZZO", "VENDITA_EURO/MQ", "AFFITTO_EURO/MQ_AL_MESE"]]
    except:
        df = pd.DataFrame({"REGIONE": [regione], 
                               "PROVINCIA": [provincia],
                               "COMUNE": [comune], 
                               "CAP": [CAP],
                               "INDIRIZZO": [None], 
                               "VENDITA_EURO/MQ": [None], 
                               "AFFITTO_EURO/MQ_AL_MESE": [None]})
        return df.loc[:,["REGIONE", "PROVINCIA", "COMUNE", "CAP", "INDIRIZZO", "VENDITA_EURO/MQ", "AFFITTO_EURO/MQ_AL_MESE"]]

Si scarica!

In [ ]:
for regione in REGIONI:
    print(regione)
    db = pd.DataFrame({"REGIONE": [], 
                       "PROVINCIA": [],
                       "COMUNE": [], 
                       "CAP": [],
                       "INDIRIZZO": [], 
                       "VENDITA_EURO/MQ": [], 
                       "AFFITTO_EURO/MQ_AL_MESE": []})
    db = db.loc[:,["REGIONE", "PROVINCIA", "COMUNE", "CAP", "INDIRIZZO", 
                   "VENDITA_EURO/MQ", "AFFITTO_EURO/MQ_AL_MESE"]]
    prov_list = get_prov(regione)
    for provincia, provincia_url in zip(prov_list["PROVINCIA"], prov_list["URL"]):
        #print("#" + provincia)
        comu_list = get_comu(provincia_url)
        for comune, comune_url in zip(comu_list["COMUNE"], comu_list["URL"]):
            #print("##" + comune)
            cap_list = get_cap(comune_url)
            for CAP, CAP_url in zip(cap_list["CAP"], cap_list["URL"]):
                #print("###" + CAP)
                db = pd.concat([db, getp(regione, provincia, comune, CAP, CAP_url)])
    db.to_csv("immobiliare_" + regione + ".tab", index = False, sep = "\t")

Abruzzo
Basilicata
Calabria


Metto tutto insieme.

In [ ]:
df = pd.read_table("immobiliare_" + REGIONI[0] + ".tab", dtype = object)
for j in range(1, len(REGIONI)):
    df = pd.concat([df, pd.read_table("immobiliare_" + REGIONI[j] + ".tab", dtype = object)])
df = df.replace(to_replace={"-": "NaN"})
df["VENDITA_EURO/MQ"] = df["VENDITA_EURO/MQ"].str.replace(".", "")
df["AFFITTO_EURO/MQ_AL_MESE"] = df["AFFITTO_EURO/MQ_AL_MESE"].str.replace(",", ".")
#df.columns = ['REGIONE', 'PROVINCIA', 'COMUNE', 'CAP', 'INDIRIZZO', 'VENDITA_EURO/MQ', 'AFFITTO_EURO/MQ_AL_MESE']
df.to_csv("immobiliare_it_prezzi_" + time.strftime("%Y_%m_%d") + ".tab", index = False, sep = "\t")

Check.

In [10]:
dtypes = {'REGIONE': object, 'PROVINCIA': object, 'COMUNE': object, 'CAP': object, 'INDIRIZZO': object, 
          'VENDITA_EURO/MQ': np.float64, 'AFFITTO_EURO/MQ_AL_MESE': np.float64}
df = pd.read_table("immobiliare_it_prezzi_" + time.strftime("%Y_%m_%d") + ".tab", dtype = dtypes)

In [11]:
df

,REGIONE,PROVINCIA,COMUNE,CAP,INDIRIZZO,VENDITA_EURO/MQ,AFFITTO_EURO/MQ_AL_MESE
0,Abruzzo,Chieti,Chieti,66100,"Corso Marrucino, 66100",1563.0,5.75
1,Abruzzo,Chieti,Chieti,66100,"Piazza dei Martiri Pennesi, 66100",NaN,NaN
2,Abruzzo,Chieti,Chieti,66100,"Piazza Giacomo Matteotti, 66100",1563.0,5.75
3,Abruzzo,Chieti,Chieti,66100,"Piazza Giuseppe Garibaldi, 66100",1463.0,5.38
4,Abruzzo,Chieti,Chieti,66100,"Piazza Malta, 66100",1563.0,5.75
5,Abruzzo,Chieti,Chieti,66100,"Piazza Paolo IV, 66100",NaN,NaN
6,Abruzzo,Chieti,Chieti,66100,"Piazza San Martino, 66100",938.0,3.25
7,Abruzzo,Chieti,Chieti,66100,"Piazza San Pio X, 66100",NaN,NaN
8,Abruzzo,Chieti,Chieti,66100,"Sopportico Educandato, 66100",1563.0,5.75
9,Abruzzo,Chieti,Chieti,66100,"Strada dei frutteti, 66100",1063.0,3.88
